# parsing nginx access log file with regex build pandas dataframe 


In [7]:
%pip install plotly
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import nginx_filter
import pandas as pd

# read all lines from the log file
with open('websrv.log', 'r') as f:
    lines = f.readlines()

parsed_lines, failed_lines = nginx_filter.parse_lines(lines)

access_df = pd.DataFrame(parsed_lines['access_log'])
error_df = pd.DataFrame(parsed_lines['error_log'])




In [9]:
# convert the timestamp to a datetime object and set it as the index.
# format: %d: day, %b: month, %Y: year, %H: hour, %M: minute, %S: second, %z: timezone
access_df['time'] = pd.to_datetime(access_df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z')

# sort the dataframe by the time
access_df =  access_df.sort_values('time', ascending=True).reindex()

#access_df.set_index('time', inplace=True)

access_df['issue'] = access_df['bad_request'].str.len() > 0



In [10]:
import pyarrow as pa

# write date frame to parquet file
access_df.to_parquet('access.parquet', index=False)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter(access_df[access_df['time'] >= '2024-11-01'],
            x="time", y="status", 
            color='issue',
            title='HTTP Status Codes over Time', 
            hover_data=['time', 'status', 'issue', 'request', 'bad_request', 'ip'],
            labels={'time': 'Time', 'status': 'HTTP Status Code'}
            )

fig_widget = go.FigureWidget(fig)
fig_widget

FigureWidget({
    'data': [{'hovertemplate': 'HTTP Status Code=%{y}<br>Time=%{x}<extra></extra>',
              'legendgroup': '200',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '200',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'bb319bbf-ae16-492c-bd3b-1e5a4eefc2af',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, 5, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 10, 22, 11, 27, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 10, 22, 11, 37, tzinfo=datetime.timezone.utc),
                          datetime.datetime(